In [15]:
import pandas as pd
import itertools
import time


def load_csv_as_transactions(csv_path):
    df = pd.read_csv(csv_path, usecols=[1], skiprows=1, names=['items'])
    return df['items'].apply(lambda x: set(x.split(' , '))).tolist()

def calculate_support(itemset, transactions):
    return sum(set(itemset).issubset(t) for t in transactions)

def find_frequent_sets(transactions, min_support, max_length=3):
    all_items = {item for transaction in transactions for item in transaction}
    subsets = []
    for length in range(1, max_length + 1):
        for combo in itertools.combinations(all_items, length):
            if (support := calculate_support(combo, transactions)) >= min_support:
                subsets.append((set(combo), support))
    return subsets


def derive_rules(frequent_sets, transactions, min_confidence):
    rules = []
    for base, base_support in frequent_sets:
        for subset in map(set, itertools.chain.from_iterable(itertools.combinations(base, r) for r in range(1, len(base)))):
            consequent = base - subset
            if len(consequent) == 0:
                continue
            subset_support = calculate_support(subset, transactions)
            confidence = base_support / subset_support if subset_support else 0
            if confidence >= min_confidence:
                rules.append((subset, consequent, confidence))
    return rules

def process_and_analyze(file_path, min_support=2, min_confidence=0.5):
    transactions = load_csv_as_transactions(file_path)
    frequent_itemsets = find_frequent_sets(transactions, min_support)
    rules = derive_rules(frequent_itemsets, transactions, min_confidence)
    return frequent_itemsets, rules

start_time = time.time()

file_paths = [
    '/Users/Punith/OneDrive/Documents/midterm/amazon_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/costco_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/best_buy_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walgreens_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walmart_data.csv'
]

for path in file_paths:
    itemsets, association_rules = process_and_analyze(path, 2, 0.5)
    print(f"Results for {path}:")
    print("Frequent Itemsets:", itemsets)
    print("Association Rules:", association_rules)
    print("\n")

print("Execution Time:", time.time() - start_time)


Results for /Users/Punith/OneDrive/Documents/midterm/amazon_data.csv:
Frequent Itemsets: [({'Diapers, Flour, Cheese, Diapers, Flour'}, 2), ({'Diapers, Flour, Cheese, Diapers'}, 53), ({'Flour, Cheese, Flour'}, 62), ({'Juice, Tea, Juice, Tea'}, 3), ({'Sugar, Sugar, Sugar'}, 6), ({'Diapers, Flour, Cheese, Diapers, Cheese'}, 2), ({'Detergent, Shampoo, Tea, Cheese, Shampoo, Shampoo'}, 3), ({'Sugar, Sugar'}, 57), ({'Flour, Cheese, Flour, Cheese'}, 3), ({'Cheese, Cheese'}, 125), ({'Tea'}, 115), ({'Detergent, Shampoo, Shampoo, Shampoo'}, 2), ({'Sugar, Diapers, Cheese, Sugar, Sugar'}, 4), ({'Flour, Juice, Juice'}, 70), ({'Detergent, Cheese, Sugar, Cheese, Detergent'}, 3), ({'Juice, Tea, Juice, Juice'}, 3), ({'Diapers, Flour, Cheese, Diapers, Diapers'}, 3), ({'Shampoo, Detergent, Detergent'}, 47), ({'Tea, Tea'}, 4), ({'Flour, Cheese, Flour, Flour'}, 2), ({'Cheese'}, 49), ({'Detergent, Tea'}, 44), ({'Detergent, Shampoo, Tea, Cheese, Shampoo'}, 47), ({'Detergent, Shampoo, Shampoo, Detergent'}, 2),

Results for /Users/akash/Documents/Data Mining Project/DataSets/walgreens_data.csv:
Frequent Itemsets: [({' First Aid Supplies,  First Aid Supplies'}, 61), ({' Band-Aids,  Toothpaste'}, 5), ({' Eye Drops'}, 16), ({' Blood Pressure Monitor,  Cold and Flu Medicine'}, 8), ({' Toothpaste '}, 5), ({' Band-Aids '}, 3), ({' Toothpaste'}, 79), ({' Blood Pressure Monitor,  Band-Aids'}, 3), ({' Thermometer,  Insect Repellent '}, 2), ({' Blood Pressure Monitor,  Blood Pressure Monitor,  Blood Pressure Monitor'}, 2), ({' First Aid Supplies '}, 55), ({'Insect Repellent'}, 50), ({'Cold and Flu Medicine'}, 193), ({' Thermometer,  Thermometer,  Thermometer'}, 5), ({' Blood Pressure Monitor,  Blood Pressure Monitor'}, 49), ({' Thermometer,  Thermometer,  Cold and Flu Medicine '}, 2), ({' First Aid Supplies,  First Aid Supplies,  First Aid Supplies'}, 5), ({' Band-Aids'}, 80), ({' Blood Pressure Monitor,  Band-Aids '}, 60), ({' Toothpaste,  Toothpaste,  Toothpaste'}, 4), ({'Blood Pressure Monitor '}, 62

Results for /Users/akash/Documents/Data Mining Project/DataSets/walmart_data.csv:
Frequent Itemsets: [({'Milk,  Milk'}, 55), ({' Dog Food,  Razors'}, 3), ({' Laundry Detergent'}, 167), ({'Razors,  Razors,  Razors'}, 4), ({' Toilet Paper '}, 7), ({' Razors'}, 56), ({' Lotion,  Lotion,  Lotion'}, 4), ({' Paper Towels'}, 54), ({' Trash Bags,  Toilet Paper'}, 6), ({' Laundry Detergent '}, 55), ({' Trash Bags '}, 59), ({' Milk,  Lotion '}, 44), ({' Trash Bags,  Laundry Detergent '}, 47), ({'Paper Towels,  Paper Towels,  Paper Towels'}, 2), ({' Milk '}, 3), ({' Paper Towels,  Paper Towels,  Paper Towels'}, 2), ({' Toilet Paper,  Milk '}, 60), ({'Paper Towels,  Paper Towels'}, 51), ({' Lotion,  Trash Bags'}, 4), ({' Paper Towels,  Paper Towels'}, 60), ({' Razors,  Milk'}, 4), ({' Milk'}, 57), ({' Dog Food,  Dog Food'}, 60), ({' Milk,  Lotion'}, 7), ({' Lotion,  Lotion'}, 45), ({' Trash Bags,  Laundry Detergent'}, 3), ({' Trash Bags'}, 117), ({' Dog Food,  Razors '}, 50), ({' Razors '}, 3), ({

In [20]:
import pandas as pd
from time import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

def read_transactions(csv_file):
    
    transaction_data = pd.read_csv(csv_file)['Filtered Transaction'].str.split(', ').tolist()
    return transaction_data

def perform_analysis(dataset_path, support_threshold, confidence_level):
    tick = time() 
    
    transactions = read_transactions(dataset_path)
    
    
    encoder = TransactionEncoder()
    transaction_matrix = encoder.fit_transform(transactions)
    transaction_df = pd.DataFrame(transaction_matrix, columns=encoder.columns_)
    
   
    frequent_sets = apriori(transaction_df, min_support=support_threshold, use_colnames=True)
    
   
    derived_rules = association_rules(frequent_sets, metric="confidence", min_threshold=confidence_level)
    
    
    dataset_name = dataset_path.split('/')[-1]  # Extracting filename for display
    print(f"Transactions from {dataset_name}:")
    for transaction in transactions:
        print(transaction)
        
    print("\nAssociation Rules derived:")
    print(derived_rules[['antecedents', 'consequents', 'support', 'confidence']])
    
    
    tock = time()
    print(f"\nProcessing time for {dataset_name}: {tock - tick:.2f} seconds\n")

def initiate_analysis():
    datasets = [
    '/Users/Punith/OneDrive/Documents/midterm/amazon_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/costco_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/best_buy_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walgreens_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walmart_data.csv'
    ]
    
    min_support = float(input("Enter minimum support value (e.g., 0.05 for 5%): "))
    min_confidence = float(input("Enter minimum confidence value (e.g., 0.5 for 50%): "))

    for path in datasets:
        perform_analysis(path, min_support, min_confidence)

if __name__ == "__main__":
    initiate_analysis()


Enter minimum support value (e.g., 0.05 for 5%):  0.02
Enter minimum confidence value (e.g., 0.5 for 50%):  0.3


Transactions from amazon_data.csv:
['Diapers', 'Flour', 'Cheese', 'Diapers']
['Flour', 'Juice', 'Juice']
['Detergent', 'Cheese', 'Sugar', 'Cheese']
['Tea']
['Sugar', 'Diapers', 'Cheese', 'Sugar']
['Detergent', 'Cheese', 'Sugar', 'Cheese']
['Diapers', 'Diapers']
['Detergent', 'Cheese', 'Sugar', 'Cheese']
['Diapers', 'Diapers']
['Tea']
['Shampoo', 'Detergent', 'Detergent']
['Diapers', 'Flour', 'Cheese', 'Diapers']
['Cheese', 'Cheese']
['Cheese', 'Cheese']
['Tea', 'Tea']
['Tea']
['Detergent', 'Shampoo', 'Shampoo']
['Tea']
['Flour', 'Juice', 'Juice']
['Cheese', 'Cheese', 'Cheese']
['Flour', 'Juice', 'Juice']
['Flour', 'Cheese', 'Flour']
['Tea']
['Shampoo', 'Detergent', 'Detergent']
['Diapers', 'Diapers']
['Detergent', 'Cheese', 'Sugar', 'Cheese']
['Sugar', 'Diapers', 'Cheese', 'Sugar']
['Sugar', 'Diapers', 'Cheese', 'Sugar']
['Cheese', 'Cheese']
['Tea']
['Shampoo', 'Detergent', 'Detergent']
['Shampoo', 'Detergent', 'Detergent']
['Cheese', 'Cheese']
['Cheese', 'Cheese']
['Cheese']
['Diapers

In [21]:
import pandas as pd
from time import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


def load_and_filter_tx(file, min_items=2):
    df = pd.read_csv(file)
    
    txs = df['Filtered Transaction'].apply(lambda x: x.split(', '))
    filtered_txs = [tx for tx in txs if len(tx) >= min_items]
    return filtered_txs


def analyze_data(file, min_sup, min_conf):
    start = time()  # Start timing

    txs = load_and_filter_tx(file)
    
    
    enc = TransactionEncoder()
    tx_array = enc.fit(txs).transform(txs)
    tx_df = pd.DataFrame(tx_array, columns=enc.columns_)
    
    
    freq_sets = apriori(tx_df, min_support=min_sup, use_colnames=True)
    
    
    ass_rules = association_rules(freq_sets, metric="confidence", min_threshold=min_conf)
    
    
    print(f"Filtered Transactions for {file.split('/')[-1]}:")
    for tx in txs:
        print(tx)
        
    print("\nGenerated Association Rules:")
    print(ass_rules[['antecedents', 'consequents', 'support', 'confidence']])
    
    
    print(f"\nTime taken for {file.split('/')[-1]}: {time() - start:.2f} seconds\n")


def run_analysis():
    datasets = {
        1: '/Users/Punith/OneDrive/Documents/midterm/amazon_data.csv',
        2: '/Users/Punith/OneDrive/Documents/midterm/costco_data.csv',
        3: '/Users/Punith/OneDrive/Documents/midterm/best_buy_data.csv',
        4: '/Users/Punith/OneDrive/Documents/midterm/walgreens_data.csv',
        5: '/Users/Punith/OneDrive/Documents/midterm/walmart_data.csv',
    }
    
    print("Select dataset(s):")
    for k, v in datasets.items():
        print(f"{k} - {v.split('/')[-1]}")
    
    choices = input("Choices (e.g., 1,3): ").split(',')
    min_sup = float(input("Minimum support (e.g., 0.05): "))
    min_conf = float(input("Minimum confidence (e.g., 0.5): "))

    for choice in choices:
        analyze_data(datasets[int(choice)], min_sup, min_conf)

if __name__ == "__main__":
    run_analysis()


Select dataset(s):
1 - amazon_data.csv
2 - costco_data.csv
3 - best_buy_data.csv
4 - walgreens_data.csv
5 - walmart_data.csv


Choices (e.g., 1,3):  3
Minimum support (e.g., 0.05):  0.02
Minimum confidence (e.g., 0.5):  0.4


Filtered Transactions for best_buy_data.csv:
['Keyboard ', ' Keyboard ', ' Keyboard ']
['Smartwatch', ' Smartwatch']
['External Hard Drive', ' External Hard Drive', ' External Hard Drive']
['Speakers ', ' Printer ', ' External Hard Drive ', ' Printer ']
['Router ', ' Charger ', ' Charger ']
['Router ', ' Charger ', ' Charger ']
['Smartwatch ', ' Printer ', ' Speakers ']
['Speakers ', ' Printer ', ' External Hard Drive ', ' Printer ']
['Keyboard ', ' Router ', ' Charger', ' Charger', ' Router ']
['Smartwatch ', ' Router ', ' Router ']
['Speakers ', ' Printer ', ' External Hard Drive ', ' Printer ', ' Speakers ']
['Keyboard ', ' Router ', ' Charger', ' Charger']
['Router ', ' Speakers ', ' Printer', ' Printer']
['External Hard Drive ', ' Charger']
['Keyboard', ' Keyboard']
['External Hard Drive ', ' Charger']
['Router ', ' Speakers ', ' Printer', ' Printer', ' Printer']
[' Router ', ' Smartwatch', ' Smartwatch']
[' Router ', ' Smartwatch', ' Smartwatch']
['External Hard Drive ', ' Charge

In [22]:

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

def fetch_transactions(csv_path):
    
    data = pd.read_csv(csv_path)['Filtered Transaction'].str.split(', ').tolist()
    return data

def fp_growth_analysis(csv_path, support_level, confidence_level):
    
    transactions = fetch_transactions(csv_path)
    
    
    encoder = TransactionEncoder()
    encoded_data = encoder.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(encoded_data, columns=encoder.columns_)
    
    
    itemsets = fpgrowth(df_encoded, min_support=support_level, use_colnames=True)
    
    
    rules = association_rules(itemsets, metric="confidence", min_threshold=confidence_level)
    
    
    print(f"Frequent Itemsets from {csv_path.split('/')[-1]}:")
    print(itemsets)
    print("\nDerived Association Rules:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence']])
    print("\n")

def run_analysis():
    
    dataset_paths = [
    '/Users/Punith/OneDrive/Documents/midterm/amazon_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/costco_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/best_buy_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walgreens_data.csv',
    '/Users/Punith/OneDrive/Documents/midterm/walmart_data.csv'
    ]
    
    
    min_support = float(input("Enter minimum support value (e.g., 0.05 for 5%): "))
    min_confidence = float(input("Enter minimum confidence value (e.g., 0.5 for 50%): "))
    
    
    for path in dataset_paths:
        fp_growth_analysis(path, min_support, min_confidence)

if __name__ == "__main__":
    run_analysis()


Enter minimum support value (e.g., 0.05 for 5%):  0.02
Enter minimum confidence value (e.g., 0.5 for 50%):  0.4


Frequent Itemsets from amazon_data.csv:
    support                           itemsets
0     0.484                           (Cheese)
1     0.198                            (Flour)
2     0.181                          (Diapers)
3     0.124                            (Juice)
4     0.262                        (Detergent)
5     0.184                            (Sugar)
6     0.272                              (Tea)
7     0.151                          (Shampoo)
8     0.127                    (Cheese, Flour)
9     0.118                  (Cheese, Diapers)
10    0.060                   (Flour, Diapers)
11    0.058                   (Sugar, Diapers)
12    0.060           (Cheese, Flour, Diapers)
13    0.058           (Cheese, Sugar, Diapers)
14    0.071                     (Flour, Juice)
15    0.053                       (Tea, Juice)
16    0.115                (Cheese, Detergent)
17    0.100                   (Detergent, Tea)
18    0.052           (Cheese, Tea, Detergent)
19    0.121         

In [19]:

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import time

def get_txs(file):
    """Fetch transactions from CSV."""
    txs = pd.read_csv(file)['Filtered Transaction'].str.split(', ').tolist()
    return txs

start_time = time.time()

def analyze_fp_growth(file, sup, conf):
    """Analyzes transactions using the FP-Growth method."""
    txs = get_txs(file)
    
    
    enc = TransactionEncoder()
    tx_data = enc.fit(txs).transform(txs)
    df_tx = pd.DataFrame(tx_data, columns=enc.columns_)
    
    
    frequent_itemsets = fpgrowth(df_tx, min_support=sup, use_colnames=True)
    
   
    assoc_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=conf)
    
    
    results = {
        'frequent_itemsets': frequent_itemsets,
        'association_rules': assoc_rules
    }
    
   
    print(f"\nFrequent Itemsets from {file.split('/')[-1]}:")
    print(results['frequent_itemsets'])
    print("\nDerived Association Rules:")
    print(results['association_rules'][['antecedents', 'consequents', 'support', 'confidence']])
    print("\n")

def run():
    
    names = ['Amazon', 'Costco', 'Best Buy', 'Walgreens', 'Walmart']
    paths = {
        '1':'/Users/Punith/OneDrive/Documents/midterm/amazon_data.csv',
        '2': '/Users/Punith/OneDrive/Documents/midterm/costco_data.csv',
        '3': '/Users/Punith/OneDrive/Documents/midterm/best_buy_data.csv',
        '4': '/Users/Punith/OneDrive/Documents/midterm/walgreens_data.csv',
        '5': '/Users/Punith/OneDrive/Documents/midterm/walmart_data.csv',
    }

    
    print("Choose dataset(s) to analyze:")
    for i, name in enumerate(names, start=1):
        print(f"{i} - {name}")
    choices = input("Enter dataset numbers (space-separated, e.g., 1 3): ").split()

    
    min_sup = float(input("\nMinimum support (e.g., 0.05): "))
    min_conf = float(input("Minimum confidence (e.g., 0.5): "))

    
    for choice in choices:
        if choice in paths:
            analyze_fp_growth(paths[choice], min_sup, min_conf)
        else:
            print(f"Invalid choice: {choice}")

if __name__ == "__main__":
    run()


Choose dataset(s) to analyze:
1 - Amazon
2 - Costco
3 - Best Buy
4 - Walgreens
5 - Walmart


Enter dataset numbers (space-separated, e.g., 1 3):  2

Minimum support (e.g., 0.05):  0.03
Minimum confidence (e.g., 0.5):  0.4



Frequent Itemsets from costco_data.csv:
    support                                           itemsets
0     0.232                                    ( Bulk Fruits )
1     0.113                                     (Bulk Fruits )
2     0.179                                   ( Optical Items)
3     0.056                                        ( Jewelry )
4     0.236                                    (Bakery Items )
5     0.121                                  ( Optical Items )
6     0.120                               ( Cleaning Supplies)
7     0.052                                     ( Bulk Fruits)
8     0.160                                          ( Tires )
9     0.105                                ( Garden Supplies )
10    0.052                                 (Garden Supplies )
11    0.111                                ( Bulk Vegetables )
12    0.053                               (Cleaning Supplies )
13    0.076                                   ( Bakery Items )
14    0.073   